In [1]:
import pandas as pd
import numpy as np


In [2]:
order_report = pd.read_excel("Company ShopX - Order Report.xlsx")
product_weight = pd.read_excel("Company ShopX - Product Weight.xlsx")
pincode_data = pd.read_excel("Company ShopX - Warehouse&Customer Pin Code and Zone details.xlsx")
invoice = pd.read_excel("Courier Company - Invoice.xlsx")
rates = pd.read_excel("Courier Company - Rates.xlsx")


In [3]:
data = pd.merge(order_report, product_weight, on="Product Code")


In [4]:
data.head()


,Order ID,Product Code,Units Ordered,Product Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350


In [18]:
data["Total Weight (kg)"] = (data["Units Ordered"] * data["Product Weight (g)"]) / 1000

In [19]:
order_weight = data.groupby("Order ID")["Total Weight (kg)"].sum().reset_index()

In [20]:
order_weight.head()

,Order ID,Total Weight (kg)
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245


In [8]:
order_weight["Weight Slab (kg)"] = ((order_weight["Total Weight (kg)"] * 2).apply(int) + 1) / 2

In [21]:
import math
def slab(weight):
    return math.ceil(weight / 0.5) * 0.5
order_weight["Weight Slab (kg)"] = order_weight["Total Weight (kg)"].apply(slab)

In [10]:
final_data = pd.merge(order_weight,invoice,on="Order ID")

In [22]:
final_data.head()

,Order ID,Total Weight (kg),Weight Slab (kg),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Zone as per Courier,Freight Type,Total Amount (Rs.),Courier Weight Slab (kg),Zone as per ShopX
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,b
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,d
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,d
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,d
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,b


In [23]:
def slab(weight):
    return math.ceil(weight / 0.5) * 0.5
final_data["Courier Weight Slab (kg)"] = final_data["Chargeable Weight"].apply(slab)

In [25]:
final_data = pd.merge(final_data,pincode_data,on=["Store House Pincode", "Customer Area Code"],
                      how="left")              

In [26]:
final_data.rename(columns={
    "Delivery Zone_x": "Zone as per Courier",
    "Delivery Zone_y": "Zone as per ShopX"
}, inplace=True)

In [27]:
final_data.head()

,Order ID,Total Weight (kg),Weight Slab (kg),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Zone as per Courier,Freight Type,Total Amount (Rs.),Courier Weight Slab (kg),Zone as per ShopX,Zone as per Courier,Zone as per ShopX
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,b,b,b
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,d,d,d
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,d,d,d
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,d,d,d
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,b,b,b


In [50]:
final_data = final_data.loc[:, ~final_data.columns.duplicated()]


In [51]:
final_data["Zone as per ShopX"] = (
    final_data["Zone as per ShopX"]
    .astype(str)
    .str.strip()
    .str.lower()
)


In [52]:
print(final_data["Zone as per ShopX"].unique())


['b' 'd' 'e']


In [53]:
def calculate_charge(row):
    
    zone = row["Zone as per ShopX"]
    slab = row["Weight Slab (kg)"]
    freight = str(row["Freight Type"])
    
    fixed_fwd = rates.loc[0, f"fwd_{zone}_fixed"]
    add_fwd = rates.loc[0, f"fwd_{zone}_additional"]
    
    additional_weight = slab - 0.5
    additional_slabs = additional_weight / 0.5
    
    forward_charge = fixed_fwd + (additional_slabs * add_fwd)
    
    if "RTO" in freight:
        fixed_rto = rates.loc[0, f"rto_{zone}_fixed"]
        add_rto = rates.loc[0, f"rto_{zone}_additional"]
        rto_charge = fixed_rto + (additional_slabs * add_rto)
        return forward_charge + rto_charge
    else:
        return forward_charge


In [54]:
final_data["Expected Charge (Rs.)"] = final_data.apply(calculate_charge, axis=1)


In [55]:
rates.head()

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [56]:
final_data = final_data.loc[:, ~final_data.columns.duplicated()]


In [57]:
def calculate_charge(row):
    
    zone = str(row["Zone as per ShopX"]).lower().strip()
    slab = row["Weight Slab (kg)"]
    freight = str(row["Freight Type"])
    
    fixed_fwd = rates.loc[0, "fwd_" + zone + "_fixed"]
    add_fwd = rates.loc[0, "fwd_" + zone + "_additional"]
    
    additional_weight = slab - 0.5
    
    forward_charge = fixed_fwd + (additional_weight / 0.5) * add_fwd
    
    if "RTO" in freight:
        fixed_rto = rates.loc[0, "rto_" + zone + "_fixed"]
        add_rto = rates.loc[0, "rto_" + zone + "_additional"]
        rto_charge = fixed_rto + (additional_weight / 0.5) * add_rto
        return forward_charge + rto_charge
    else:
        return forward_charge


In [58]:
final_data["Expected Charge (Rs.)"] = final_data.apply(calculate_charge, axis=1)


In [59]:
print(final_data.columns)


Index(['Order ID', 'Total Weight (kg)', 'Weight Slab (kg)',
       'AWB Code (Airway Bill Number) ', 'Chargeable Weight',
       'Store House Pincode', 'Customer Area Code', 'Zone as per Courier',
       'Freight Type', 'Total Amount (Rs.)', 'Courier Weight Slab (kg)',
       'Zone as per ShopX', 'Expected Charge (Rs.)', 'Difference (Rs.)',
       'Status'],
      dtype='object')


In [60]:
final_data["Difference (Rs.)"] = final_data["Total Amount (Rs.)"] - final_data["Expected Charge (Rs.)"]


In [61]:
def status(diff):
    if diff == 0:
        return "Correct"
    elif diff > 0:
        return "Overcharged"
    else:
        return "Undercharged"

final_data["Status"] = final_data["Difference (Rs.)"].apply(status)


In [62]:
summary = final_data.groupby("Status").agg(
    Count=("Order ID", "count"),
    Amount=("Difference (Rs.)", "sum")
).reset_index()

summary


,Status,Count,Amount
0,Correct,30,0.0
1,Overcharged,1047,53002.0
2,Undercharged,45,-1651.2


In [64]:
final_data["Difference (Rs.)"] = (
    final_data["Total Amount (Rs.)"] - 
    final_data["Expected Charge (Rs.)"]
)


In [65]:
summary_list = []

for status in ["Correct", "Overcharged", "Undercharged"]:
    
    temp = final_data[final_data["Status"] == status]
    
    count = len(temp)
    
    if status == "Correct":
        amount = temp["Total Amount (Rs.)"].sum()
    
    elif status == "Overcharged":
        amount = temp["Difference (Rs.)"].sum()
    
    else:  # Undercharged
        amount = abs(temp["Difference (Rs.)"].sum())
    
    summary_list.append([status, count, amount])


summary = pd.DataFrame(
    summary_list,
    columns=["Status", "Count", "Amount (Rs.)"]
)

summary


,Status,Count,Amount (Rs.)
0,Correct,30,2126.8
1,Overcharged,1047,53002.0
2,Undercharged,45,1651.2


In [67]:
final_data["Difference (Rs.)"] = (
    final_data["Total Amount (Rs.)"] - 
    final_data["Expected Charge (Rs.)"]
)


In [68]:
summary_list = []

for status in ["Correct", "Overcharged", "Undercharged"]:
    
    temp = final_data[final_data["Status"] == status]
    count = len(temp)
    
    if status == "Correct":
        # Sum of billed amount
        amount = temp["Total Amount (Rs.)"].sum()
    
    elif status == "Overcharged":
        # Positive difference
        amount = temp["Difference (Rs.)"].sum()
    
    else:  # Undercharged
        # Make it positive
        amount = abs(temp["Difference (Rs.)"].sum())
    
    summary_list.append([status, count, amount])

summary = pd.DataFrame(
    summary_list,
    columns=["Status", "Count", "Amount (Rs.)"]
)

summary


,Status,Count,Amount (Rs.)
0,Correct,30,2126.8
1,Overcharged,1047,53002.0
2,Undercharged,45,1651.2


In [69]:
with pd.ExcelWriter("Anjul_Logistic.xlsx") as writer:
    final_data.to_excel(writer, sheet_name="Order Level Data", index=False)
    summary.to_excel(writer, sheet_name="Summary", index=False)
